<a href="https://colab.research.google.com/github/rtldr/DeepDecompiler/blob/master/DeepDecompiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
!pip3 install pyelftools

In [10]:
DATA_FOLDER_PATH = '/content/drive/MyDrive/colab_files/final_project/data'
DATA_V2_FOLDER_PATH = '/content/drive/MyDrive/colab_files/final_project/data_v2'

In [ ]:
# Simple asm tokenizer - TODO maybe look into better ways of doing this
from elftools.elf.elffile import ELFFile

NUM_TOKENS = 2**8 + 2
EOF = 256
PAD = 257

def tokenize_asm_file(filename):
    with open(filename, "rb") as f:
        print('here1')
        elf_file = ELFFile(f)
        print('here2')
        for section in elf_file.iter_sections():
            print('here3')
            if section.name == '.text':
                bytes_list = list(section.data())
    # Add EOF
    bytes_list.append(EOF)
    return bytes_list
    
if __name__ == "__main__":
    m = 0
    count = 0
    for i in range (1024):
        print(i)
        tokenized = tokenize_asm_file(DATA_V2_FOLDER_PATH + "/asm/" + str(i))
        if len(tokenized) < 100:
            count += 1
            m = max(m, len(tokenized))
    print(m)
    print(count)


0
